In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assitant"

In [ ]:
def message_apt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
message_apt("what is today date")

## User Interface

In [ ]:
def shout(text):
    return text.upper()


gr.Interface(fn=shout, inputs="textbox", outputs="textbox", allow_flagging="never").launch()

In [ ]:
view = gr.Interface(
    fn=message_apt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    allow_flagging="never").launch()

In [ ]:
view = gr.Interface(
    fn=message_apt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never").launch()

In [ ]:
def stream_gpt(promt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": promt},
    ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
).launch()

In [ ]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_content(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [ ]:
def stream_brochure(company_name, url, model):
    prompt=f"Please generate a company borchure for {company_name}. Here is their landing page: \n"
    prompt+=Website(url).get_content()
    model="gpt-4o-mini"
    result=stream_gpt(prompt)
    for chunk in result:
        yield chunk

In [ ]:
gr.Interface(fn=stream_brochure,
             inputs=[gr.Textbox(label="Company name:"),
                     gr.Textbox(label="Landing page URL:")],
             outputs=[gr.Markdown(label="Brochure:")],
             allow_flagging="never").launch()